In [22]:
import pandas as pd

# 1. Preparación de Datos
df = pd.read_csv('Customer_Data.csv')
df = df[['Flights Booked', 'Education']]

# 2. Análisis Descriptivo
estadisticas = df.groupby('Education')['Flights Booked'].describe()
print(estadisticas)



                         count      mean       std  min  25%  50%  75%   max
Education                                                                   
Bachelor              252567.0  4.110288  5.221671  0.0  0.0  1.0  8.0  21.0
College               102260.0  4.169744  5.246040  0.0  0.0  1.0  8.0  21.0
Doctor                 17731.0  4.175512  5.256971  0.0  0.0  1.0  8.0  21.0
High School or Below   18915.0  4.176209  5.239267  0.0  0.0  1.0  8.0  21.0
Master                 12287.0  4.200700  5.213956  0.0  0.0  1.0  8.0  21.0


### Conclusión: 

#### Asimetría en Percentiles:

Los percentiles (25%, 50%, 75%) para todos los grupos de educación están concentrados en los valores bajos:
- 25%: 0
- 50% (mediana): 1
- 75%: 8
Esto sugiere que hay una gran acumulación de datos cerca de los valores bajos, lo que podría indicar una distribución sesgada a la derecha.

#### Desviación Estándar vs Media:

La desviación estándar es mayor que la media en todos los grupos de educación. Esto indica una dispersión considerable, característica de distribuciones sesgadas.

#### Valor Máximo:

Los valores máximos son significativamente más altos que la media (21 vs ~4.11). Esto sugiere la presencia de valores extremos (outliers) que son típicos de una distribución sesgada.

### Pruebas Estadísticas:

In [18]:
import pandas as pd
import scipy.stats as stats

# Supongamos que df ya está definido

# Agrupar datos por nivel de educación
bachelor = df[df['Education'] == 'Bachelor']['Flights Booked']
college = df[df['Education'] == 'College']['Flights Booked']
doctor = df[df['Education'] == 'Doctor']['Flights Booked']
high_school = df[df['Education'] == 'High School or Below']['Flights Booked']
master = df[df['Education'] == 'Master']['Flights Booked']

# Realizar prueba de Kruskal-Wallis
kruskal_test = stats.kruskal(bachelor, college, doctor, high_school, master)
print(f"Estadístico H: {kruskal_test.statistic}, Valor p: {kruskal_test.pvalue}")


Estadístico H: 17.532618623121756, Valor p: 0.0015225470219889365


- Estadísticas Descriptivas: Para cada nivel de educación, muestran cómo se distribuyen los vuelos reservados.
- ANOVA: Compara las medias de vuelos reservados entre los diferentes niveles de educación para ver si hay diferencias significativas.
- Tukey HSD: Similar al anterior, determina entre qué grupos de educación existen diferencias significativas en el número de vuelos reservados.

In [15]:
# 3. Prueba Estadística
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd


# Agrupar por nivel educativo y calcular estadísticas descriptivas
stats = df.groupby('Education')['Flights Booked'].describe()

# Calcular percentiles adicionales
percentiles = df.groupby('Education')['Flights Booked'].quantile([0.25, 0.5, 0.75]).unstack()

# Combinar las estadísticas descriptivas y los percentiles
stats = stats.join(percentiles, rsuffix='_percentile')
print("\nEstadísticas descriptivas & Percentiles:")
print(stats)

# Extraer las listas de vuelos reservados por cada nivel educativo
education_levels = df['Education'].unique()
groups = [df[df['Education'] == level]['Flights Booked'] for level in education_levels]

# Realizar la prueba ANOVA
anova_result = f_oneway(*groups)
print("\nPrueba ANOVA:")
print(anova_result)

# Realizar la prueba de Tukey HSD
tukey = pairwise_tukeyhsd(endog=df['Flights Booked'], groups=df['Education'], alpha=0.05)
print("\nPrueba Tukey HSD:")
print(tukey)



Estadísticas descriptivas & Percentiles:
                         count      mean       std  min  25%  50%  75%   max  \
Education                                                                      
Bachelor              252567.0  4.110288  5.221671  0.0  0.0  1.0  8.0  21.0   
College               102260.0  4.169744  5.246040  0.0  0.0  1.0  8.0  21.0   
Doctor                 17731.0  4.175512  5.256971  0.0  0.0  1.0  8.0  21.0   
High School or Below   18915.0  4.176209  5.239267  0.0  0.0  1.0  8.0  21.0   
Master                 12287.0  4.200700  5.213956  0.0  0.0  1.0  8.0  21.0   

                      0.25  0.5  0.75  
Education                              
Bachelor               0.0  1.0   8.0  
College                0.0  1.0   8.0  
Doctor                 0.0  1.0   8.0  
High School or Below   0.0  1.0   8.0  
Master                 0.0  1.0   8.0  

Prueba ANOVA:
F_onewayResult(statistic=3.57895773849834, pvalue=0.006352981736019541)

Prueba Tukey HSD:
           

## Interpretación

### 1. Estadísticas Descriptivas & Percentiles:
Estas tablas muestran estadísticas descriptivas como el recuento (count), la media (mean), la desviación estándar (std), los valores mínimo (min) y máximo (max), así como los percentiles (25%, 50%, 75%) para diferentes niveles de educación (Bachelor, College, Doctor, High School or Below, Master).

Por ejemplo, para el nivel de educación "Bachelor":

Hay 252,567 observaciones.
La media de la medida es aproximadamente 4.11, con una desviación estándar de aproximadamente 5.22.
Los valores mínimo y máximo son 0 y 21, respectivamente.
Los percentiles 25%, 50% (mediana) y 75% son 0, 1, y 8, respectivamente.
Esto se repite para cada grupo de nivel educativo.



### 2. Prueba ANOVA:
El resultado de la prueba ANOVA (Análisis de Varianza) indica si hay diferencias significativas en la medida numérica entre los diferentes grupos de educación. En este caso, el estadístico F es 3.5789 y el valor p es 0.00635. Un valor p menor que el nivel de significancia (generalmente 0.05) sugiere que hay diferencias estadísticamente significativas entre al menos algunos de los grupos.

### 3. Prueba Tukey HSD:
La prueba Tukey HSD (Diferencia Mínima Significativa de Tukey) se utiliza típicamente después de una prueba ANOVA para determinar qué grupos específicos difieren entre sí. La tabla muestra las diferencias medias (meandiff) entre pares de grupos, los intervalos de confianza (lower y upper) para esas diferencias y si se rechaza la hipótesis nula de igualdad de medias para cada par de grupos (reject).

### Conclusión: 
Se rechaza la hipótesis nula de igualdad de medias entre los grupos "Bachelor" y "College" (reject=True), ya que la diferencia media (meandiff) es significativa y el valor de p ajustado (p-adj) es menor que el nivel de significancia (0.05).
En resumen, estos resultados indican que hay diferencias significativas entre al menos algunos de los grupos de educación en términos de la medida numérica analizada, según la prueba ANOVA y la prueba Tukey HSD.